- 용인시 행정동 죽전1동에 대한 p-median을 통해 최적의 입지 갯수 4개에 대한 최적의 입지 searching 
- 기준은 소비자 총 통행거리와 설치비용을 최소화하여 최적화하는 방식

In [1]:
import pandas as pd
import numpy as np
import random
import gurobipy as gp
from gurobipy import GRB


### Input Data

In [2]:
path = 'C:/bigcon'
jukjun1_od = pd.read_csv(path + 'output\data\checkpoint\jukjun1_od_mat.csv', encoding = 'utf-8')

In [3]:
# demand 따로 추출 
jukjun1_demand = jukjun1_od[['앱실행수']]
jukjun1_demand

,앱실행수
0,11015.605
1,10843.670
2,7606.771
3,6591.808
4,5416.644
5,5143.311
6,5127.690
7,4517.616
8,4352.650
9,4199.244


In [4]:
jukjun1_od_drop = jukjun1_od.drop(['앱실행수'],axis = 1)
jukjun1_od_drop

,Unnamed: 0,꽃메마을 한라신영프로방스,꽃메마을 현대홈타운 (4차 4단지),꽃메마을 현대홈타운 (4차 3단지),꽃메마을 극동스타클래스,꽃메마을 금강아파트,꽃메마을 아이파크,꽃메마을 한라프로방스 Ⅱ,도담마을 푸르지오아파트(601-602동),꽃메마을 현대홈타운 (4차2단지),...,새터마을 푸르지오아파트,대지마을 중앙하이츠빌,죽전 현대1차아파트,대지마을 현대홈타운 3차 2단지,내대지마을 정석베르데파크아파트,내대지마을 광명샤인빌,내대지마을 진흥더블파크,새터마을 죽전힐스테이트,내대지마을 건영캐스빌,죽전1동
0,0,1599.296758,1426.537077,1304.948730,1232.246850,1369.660586,1158.431465,1162.672420,1301.883713,1158.115519,...,296.735328,496.692844,780.302145,660.667641,601.888742,925.502337,843.203121,183.828968,365.641395,788.311686
1,1,369.724799,216.192061,239.996248,536.015489,140.907708,386.591958,290.921814,112.858602,79.915140,...,1018.752461,899.778474,1603.982867,1003.596894,1514.319465,1776.032860,1720.416193,1126.933451,1506.391192,797.822254
2,2,413.864442,405.726408,503.507075,803.540744,266.807665,650.172693,554.867425,183.651042,255.683221,...,1119.866513,866.835802,1764.477628,909.730348,1654.107927,1939.095474,1876.596949,1145.891706,1596.250441,683.725501
3,3,2473.704644,2250.338839,2062.479264,1781.616029,2274.008708,1838.873718,1910.333662,2252.442775,2104.036853,...,1295.918198,1801.725268,591.860260,1999.812646,755.075528,427.686838,508.730408,1528.663501,1018.972518,2078.501148
4,4,1960.326663,1737.468548,1550.736288,1279.020345,1759.785573,1328.008044,1397.509631,1738.678339,1590.818874,...,835.029507,1356.794125,222.791413,1570.030489,391.037980,248.063915,271.731225,1122.670454,714.117965,1617.542631
5,5,1102.848384,913.426727,754.791651,465.726268,1027.260306,626.064625,712.846566,1079.160102,1014.688043,...,1246.754559,1523.088010,1389.883225,1721.159747,1417.440591,1511.513926,1499.991190,1558.711014,1610.161620,1592.525355
6,6,677.779493,668.035089,734.222135,993.478061,531.070615,833.199132,744.174520,440.707903,446.217714,...,1068.852279,704.153736,1755.944144,692.770742,1624.236423,1930.226709,1860.679614,1013.064690,1514.839462,464.769057
7,7,963.377805,881.144305,870.291266,1029.041288,764.981345,877.044001,811.559658,672.550788,592.296414,...,780.849668,346.872883,1484.722021,349.852687,1336.036764,1655.230905,1580.368882,663.273331,1187.427928,129.607739
8,8,564.031074,339.108347,152.089622,234.544164,391.239714,84.652324,21.399329,411.715778,321.774652,...,918.212392,965.619932,1404.637185,1122.927220,1341.972182,1570.421742,1523.808635,1116.839506,1394.131223,953.133975
9,9,407.944460,185.550810,63.126508,372.227884,236.142991,241.581639,156.417057,272.236565,230.600083,...,1024.805335,996.139990,1549.473111,1127.264412,1478.214710,1717.369341,1668.117394,1185.773394,1508.764654,935.956447


In [5]:
# 출발지 기호 변경
depart_oldname = jukjun1_od_drop['Unnamed: 0'].unique().tolist()
depart_newname = [] # 바뀌는 값
for i in range(1,len(depart_oldname)+1):
    depart_newname.append('D' + str(i))

jukjun1_od_drop['수요지'] = jukjun1_od_drop['Unnamed: 0'].replace(depart_oldname,depart_newname) # 수요지 열 재생성
jukjun1_od_drop = jukjun1_od_drop.set_index(['수요지'])
jukjun1_od_drop.drop(['Unnamed: 0'],axis=1, inplace=True)

# 도착지 기호 변경
arrive_oldname = jukjun1_od_drop.columns.unique().tolist()
arrive_newname = [] # 바뀌는 값
for i in range(1,len(arrive_oldname)+1):
    arrive_newname.append('A' + str(i))

jukjun1_od_drop.columns = arrive_newname
jukjun1_od_drop

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A39,A40,A41,A42,A43,A44,A45,A46,A47,A48
수요지,,,,,,,,,,,,,,,,,,,,,
D1,1599.296758,1426.537077,1304.948730,1232.246850,1369.660586,1158.431465,1162.672420,1301.883713,1158.115519,1198.526129,...,296.735328,496.692844,780.302145,660.667641,601.888742,925.502337,843.203121,183.828968,365.641395,788.311686
D2,369.724799,216.192061,239.996248,536.015489,140.907708,386.591958,290.921814,112.858602,79.915140,111.210086,...,1018.752461,899.778474,1603.982867,1003.596894,1514.319465,1776.032860,1720.416193,1126.933451,1506.391192,797.822254
D3,413.864442,405.726408,503.507075,803.540744,266.807665,650.172693,554.867425,183.651042,255.683221,173.912546,...,1119.866513,866.835802,1764.477628,909.730348,1654.107927,1939.095474,1876.596949,1145.891706,1596.250441,683.725501
D4,2473.704644,2250.338839,2062.479264,1781.616029,2274.008708,1838.873718,1910.333662,2252.442775,2104.036853,2177.973205,...,1295.918198,1801.725268,591.860260,1999.812646,755.075528,427.686838,508.730408,1528.663501,1018.972518,2078.501148
D5,1960.326663,1737.468548,1550.736288,1279.020345,1759.785573,1328.008044,1397.509631,1738.678339,1590.818874,1665.448888,...,835.029507,1356.794125,222.791413,1570.030489,391.037980,248.063915,271.731225,1122.670454,714.117965,1617.542631
D6,1102.848384,913.426727,754.791651,465.726268,1027.260306,626.064625,712.846566,1079.160102,1014.688043,1087.871815,...,1246.754559,1523.088010,1389.883225,1721.159747,1417.440591,1511.513926,1499.991190,1558.711014,1610.161620,1592.525355
D7,677.779493,668.035089,734.222135,993.478061,531.070615,833.199132,744.174520,440.707903,446.217714,383.924388,...,1068.852279,704.153736,1755.944144,692.770742,1624.236423,1930.226709,1860.679614,1013.064690,1514.839462,464.769057
D8,963.377805,881.144305,870.291266,1029.041288,764.981345,877.044001,811.559658,672.550788,592.296414,575.906090,...,780.849668,346.872883,1484.722021,349.852687,1336.036764,1655.230905,1580.368882,663.273331,1187.427928,129.607739
D9,564.031074,339.108347,152.089622,234.544164,391.239714,84.652324,21.399329,411.715778,321.774652,398.442969,...,918.212392,965.619932,1404.637185,1122.927220,1341.972182,1570.421742,1523.808635,1116.839506,1394.131223,953.133975


In [7]:
# 죽전1동 arc 및 demand(P/T) dict
dic = {}

for i in jukjun1_od_drop.index:
    for name in jukjun1_od_drop.columns:
        key = (i, name)
        value = jukjun1_od_drop.loc[i, name]
        dic[key] = value

In [8]:
dic['D1','A1']

1599.2967575044102

In [9]:
# 수요지와 도착지 
demand = {i for (i,j) in dic.keys()}
print("수요지 =",demand)

arrival = {j for (i,j) in dic.keys()}
print("전기차 충전소 후보지 =",arrival)

수요지 = {'D13', 'D5', 'D19', 'D2', 'D4', 'D18', 'D9', 'D6', 'D7', 'D16', 'D11', 'D14', 'D17', 'D20', 'D3', 'D10', 'D12', 'D15', 'D1', 'D8'}
전기차 충전소 후보지 = {'A10', 'A40', 'A47', 'A14', 'A25', 'A22', 'A33', 'A38', 'A30', 'A36', 'A17', 'A43', 'A3', 'A32', 'A8', 'A46', 'A2', 'A6', 'A11', 'A29', 'A24', 'A16', 'A35', 'A26', 'A12', 'A34', 'A13', 'A31', 'A48', 'A21', 'A4', 'A18', 'A37', 'A1', 'A20', 'A44', 'A9', 'A39', 'A28', 'A41', 'A45', 'A42', 'A19', 'A7', 'A5', 'A23', 'A27', 'A15'}


In [14]:
# 출발지 기호 변경
depart_oldname = jukjun1_demand.index.unique().tolist()
depart_newname = [] # 바뀌는 값
for i in range(1,len(depart_oldname)+1):
    depart_newname.append('D' + str(i))

jukjun1_demand.index = depart_newname 
jukjun1_demand.loc['D1','앱실행수']

11015.605

In [15]:
# 수요량 parameter

h = {}

for i in jukjun1_demand.index:
    for name in jukjun1_demand.columns:
        key = i
        value = jukjun1_demand.loc[i,name]
        h[key] = value

h['D1']

11015.605

### Model deployment

In [16]:
model = gp.Model("electric_p_median")

# variables
x = model.addVars(arrival, vtype = GRB.BINARY, name='x_j')
y = model.addVars(demand, arrival, vtype = GRB.BINARY, name='y_ij')

In [17]:
# objective function
obj = gp.quicksum(h[i] * dic[i,j] * y[i,j] for i in demand for j in arrival)
        
model.setObjective(obj, GRB.MINIMIZE)

In [18]:
# constraints

model.addConstrs(gp.quicksum(y[i,j] for j in arrival) == 1 for i in demand)
model.addConstrs(gp.quicksum(x[j] for j in arrival) == 4 for i in demand)
model.addConstrs(y[i,j] <= x[j] for i in demand for j in arrival)

{('D13', 'A10'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A40'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A47'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A14'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A25'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A22'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A33'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A38'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A30'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A36'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A17'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A43'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A3'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A32'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A8'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A46'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A2'): <gurobi.Constr *Awaiting Model Update*>,
 ('D13', 'A6'): <

In [19]:
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1000 rows, 1008 columns and 3840 nonzeros
Model fingerprint: 0x8fed94cd
Variable types: 0 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+04, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 19 rows and 0 columns
Presolve time: 0.01s
Presolved: 981 rows, 1008 columns, 2928 nonzeros
Variable types: 0 continuous, 1008 integer (1008 binary)
Found heuristic solution: objective 4.131039e+07

Root relaxation: objective 2.971909e+07, 264 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2.971909e+07 2.9719e+07  0.00%     -    0s

Explored 1 nodes (264 si

In [20]:
for v in model.getVars():
    if v.X != 0:
        print("%s : %f" % (v.Varname, v.X))

print(model.objVal)

x_j[A10] : 1.000000
x_j[A46] : 1.000000
x_j[A6] : 1.000000
x_j[A44] : 1.000000
y_ij[D13,A6] : 1.000000
y_ij[D5,A44] : 1.000000
y_ij[D19,A46] : 1.000000
y_ij[D2,A10] : 1.000000
y_ij[D4,A44] : 1.000000
y_ij[D18,A44] : 1.000000
y_ij[D9,A6] : 1.000000
y_ij[D6,A6] : 1.000000
y_ij[D7,A10] : 1.000000
y_ij[D16,A10] : 1.000000
y_ij[D11,A44] : 1.000000
y_ij[D14,A6] : 1.000000
y_ij[D17,A46] : 1.000000
y_ij[D20,A46] : 1.000000
y_ij[D3,A10] : 1.000000
y_ij[D10,A6] : 1.000000
y_ij[D12,A46] : 1.000000
y_ij[D15,A46] : 1.000000
y_ij[D1,A46] : 1.000000
y_ij[D8,A10] : 1.000000
29719087.98394493


In [22]:
# 죽전1동 전기차 충전소 후보지 4곳 선정

arrival_name_df = pd.DataFrame({'old' : arrive_oldname,
                                'new' : arrive_newname})
arrival_name_df[(arrival_name_df['new'] == 'A10') | (arrival_name_df['new'] == 'A46')|
               (arrival_name_df['new'] == 'A6') | (arrival_name_df['new'] == 'A44')]

,old,new
5,꽃메마을 아이파크,A6
9,도담마을 7단지 뜨리에체아파트,A10
43,내대지마을 광명샤인빌,A44
45,새터마을 죽전힐스테이트,A46
